In [1]:
# import packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq
import itertools
import sys
import read_group as rg


In [2]:
# set environment folders
data_path = "/media/manbobo/UltraP/Research/USEDU/data"

In [ ]:
# test df
# dt1 = {field_name: rg.make_default_list(field_default, 4, 20) for (field_name, field_default) in itertools.zip_longest(rg.edu_var, rg.edu_default)}
dt1 = {"A": rg.make_default_list("amd", 4, 100), "B": rg.make_default_list("nvidia", 4, 100), "C": rg.make_default_list(date(2077,9,10), 4, 100)}
df1 = pd.DataFrame(dt1)


print(dt1)
print(df1)

In [4]:
# test storing and saving df read from data

edus = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_education.parquet".format(data_path=data_path))
# display edus metadata
group0 = edus.read_row_group(0).to_pandas()

print(group0.head())

     user_id                university_raw               university_name  \
0  130452445  Purdue University Fort Wayne  Purdue University Fort Wayne   
1  130452445        Dale Carnegie Training        dale carnegie training   
2  617479683   University of North Florida   University of North Florida   
3  617479683      Florida State University       Fachhochschule Dortmund   
4  617479683      Florida State University       Fachhochschule Dortmund   

   education_number   startdate     enddate    degree      field  \
0                 1  2006-01-01  2010-01-31  Bachelor  Marketing   
1                 2  2013-01-01  2013-01-01     empty      empty   
2                 3  1990-01-01  1992-01-31    Master  Education   
3                 2  1983-01-01  1987-01-31  Bachelor  Education   
4                 1  1983-01-01  1987-01-01  Bachelor   Business   

                        degree_raw  \
0              Bachelor of Science   
1  Leadership Development Training   
2      Master of Edu

In [5]:
# print(group0.columns)
print(type(group0.at[0, "startdate"]))
# print(datetime.strptime("2010-09-10", "%Y-%m-%d"))
# print(datetime.strptime(group0.at[0, "startdate"], "%Y-%m-%d"))

<class 'datetime.date'>


In [23]:
dt = {field_name: rg.make_default_list(field_default, 2, 2) for (field_name, field_default) in itertools.zip_longest(rg.pos_var, rg.pos_default)}
dt["user_id"] = [438935973, 339609382]
df = pd.DataFrame(dt)

col_names = df.columns.to_list()
col_names.remove("user_id")
col_names.remove("updated")

for col in col_names:
    if (col == "startdate") | (col == "enddate"):
        df.at[0, col][0] = datetime.strptime(group0.at[0, col], "%Y-%m-%d")
    else:
        df.at[0, col][0] = group0.at[0, col]

print(df)

df.to_parquet(path = "./test.parquet", index = False)

                position_id                            company_raw  \
0  [7486565030000009884, 0]  [IM SAFE Child Advocacy Center, none]   
1                    [0, 0]                           [none, none]   

  company_linkedin_url                        company_cleaned  \
0         [None, none]  [im safe child advocacy center, none]   
1         [none, none]                           [none, none]   

                               location_raw                    region  \
0  [Fenwick, Michigan, United States, none]  [Northern America, none]   
1                              [none, none]              [none, none]   

                 country             state  \
0  [United States, none]  [Michigan, none]   
1           [none, none]      [none, none]   

                              metro_area                          startdate  \
0  [michigan nonmetropolitan area, none]  [2021-05-01 00:00:00, 2077-09-10]   
1                           [none, none]           [2077-09-10, 2077-09-10] 

In [ ]:
# read the number of group numbers in all the files
file_list = os.listdir("{dir_path}/US_EDUC".format(dir_path=data_path))

row_group_list = []

for file in file_list:
    edus = pq.ParquetFile("{data_path}/US_EDUC/{file}".format(data_path=data_path, file = file))
    row_group_list.append(edus.num_row_groups)

print(row_group_list)
print(file_list)

In [ ]:

os.listdir("{data_path}/US_EDUC".format(data_path=data_path))

In [ ]:
# read user id dataset as pd dataframe
ids = pq.read_table("{data_path}/unique_user_id_US_EDUC.parquet".format(data_path=data_path)).to_pandas()
ids.sort_values("user_id", inplace=True)
# sort user id, later would useful in filtering
# ids = ids.sort_values("user_id")
# min_id = ids.column("user_id")[0].as_py()
# max_id = ids.column("user_id")[-1].as_py()
# ids.column("user_id").index(697508942)
print(ids)

In [ ]:
print(ids["user_id"])
target_id = set(ids["user_id"])
print(len(target_id))

In [ ]:
# check min and max id
print("min id is,", min_id)
print("max id is,", max_id)
print(ids.column("user_id")[100000])

In [ ]:
# read education
edus = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_education.parquet".format(data_path=data_path))
# display edus metadata
group0 = edus.read_row_group(0).to_pandas()
# group0 = group0[(group0["user_id"] >= min_id) & (group0["user_id"] <= max_id)]
remove_id = pd.unique(group0[group0["education_number"] > 6]["user_id"])
group_data = group0[~group0["user_id"].isin(remove_id)]

group_data = group_data.sort_values(["user_id", "enddate"])

group_data.index = range(0, group_data.shape[0])

print(group0.shape)

In [ ]:
# try extracting common ids in group data and ids
common_id = set(group0["user_id"]).intersection(set(ids["user_id"]))
print(len(common_id))

In [ ]:
# tbid = group0[group0["user_id"] == 1858152]
# access_row = rg.find_index(group_data, "user_id", 1858152)
print(group_data.iloc[827:830,:])

# print(rg.find_index(group_data, "university_raw", "The University of Manchester"))
# print(group_data.index)
# print(group_data["user_id"])

In [ ]:
print(group0.index)

In [ ]:
# get the number of people with insane education history

off_df = group0[group0["education_number"] > 6]
len(pd.unique(off_df["user_id"]))

In [ ]:
# print(group0.head())
# drop the outlier
# group0 = group0[group0["user_id"] != 697508942]
# get the maximum education number
# max(group0["education_number"])
# print(group0[group0["education_number"] == 8])
# print(group0.loc[359753])
print(group0[group0["user_id"] == 355326980])

In [ ]:
# check time of constructing and updating one edu instance
edu_test = rd.edu()
edu_test.update_value(group0.loc[0])
print(edu_test)

In [ ]:
# test value access in pandas.df
df = pd.DataFrame(np.arange(12).reshape(3, 4),

                  columns=['A', 'B', 'C', 'D'])

print(df)

df[0:2]


In [ ]:
# check group0 properties
# group0.slice(length=10).sort_by([("user_id","ascending"),("enddate","ascending")])
# group0.filter(group0.field("user_id") == 323)
group0.field("user_id")
expr = pc.field("user_id") < 323
sub0 = group0.filter(mask = expr, null_selection_behavior = "drop")
sub0.num_rows

In [ ]:
print(group0.column("user_id").index(111))
print(group0.column("user_id").index(130452445))

In [ ]:
id_expr = pc.field("user_id") == 130452445
ids.filter(id_expr)

In [ ]:
# test df construction time

numrows = ids.num_rows
dt = {"user_id": ids.column("user_id"), "user_prof": pd.Series([rd.user()] * numrows), "skill": pd.Series([rd.skill()] * numrows), "edu1": pd.Series([rd.edu()] * numrows), "edu2": pd.Series([rd.edu()] * numrows), "edu3": pd.Series([rd.edu()] * numrows), "edu4": pd.Series([rd.edu()] * numrows), "pos1": pd.Series([rd.pos()] * numrows), "pos2": pd.Series([rd.pos()] * numrows), "pos3": pd.Series([rd.pos()] * numrows), "pos4": pd.Series([rd.pos()] * numrows) }
df = pd.DataFrame(dt)
# tb = pa.Table.from_pydict(dt)

In [ ]:
# print(df.head())
print(df.dtypes)

In [ ]:
# check user dataset
prof = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_part_1_0_249.parquet".format(data_path=data_path))
prof0 = prof.read_row_group(5).to_pandas()
prof0[prof0["user_id"] == 697508942]





In [ ]:
# test time of one iteration before df assignment
expr = pc.field("user_id") == 600272782
thisuser = prof0.filter(mask = expr, null_selection_behavior = "drop")

test_prof = rd.user()
test_prof.update_value(thisuser.take([0]).to_pandas())



In [ ]:
# test df assignment time
id_index = ids.column("user_id").index(3655629).as_py()
print(id_index)
print(df.loc[id_index, "user_prof"])


In [ ]:
df.loc[id_index, "user_prof"].update_value(thisuser.take([0]).to_pandas())
print(df.loc[id_index, "user_prof"])

In [ ]:
# test time of just df assignment
df.loc[id_index, "user_prof"] = test_prof

In [ ]:
prof0.slice(length=5).column("user_id")

In [ ]:
# check user_position dataset
pos = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_position_part_1_00_0_249.parquet".format(data_path=data_path))
pos0 = pos.read_row_group(0)
pos0.schema
pos0.num_rows

In [ ]:
print(pos0.slice(length=1).column(0)[0])
print(pos0.column_names[0])
print(pos0.column("user_id")[0])

In [ ]:
# pos0.select(["title_raw", "role_k1500", "job_category", "role_k50", "role_k150"])
for (col_name, col_type) in itertools.zip_longest(pos0.schema.names, pos0.schema.types):
    print(col_name,",", col_type)

In [ ]:
# check user_skill dataset
skill = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_skill_part_0000_0049.parquet".format(data_path=data_path))
skill0 = skill.read_row_group(0)
skill0.num_columns
skill0.column("user_id")[0]

In [ ]:
skill0.slice(length=3)

In [ ]:
# test mutating df
df1 = pd.DataFrame({"a":[1,2,3, 0], "b":[2,3,4,1], "c":pd.Series([pd.NA] * 4), "d":[7,7,8,6], "e": ["drop", "may_drop", "no_drop", "drop"]} )

print(df1)

df1.iloc[]

In [ ]:
for i in range(3,-1,-1):
    print(i)

In [ ]:
user_var= ("user_id",
    "firstname",
    "lastname",
    "fullname",
    "f_prob", 
    "m_prob", 
    "white_prob", 
    "black_prob", 
    "api_prob", 
    "hispanic_prob", 
    "native_prob", 
    "multiple_prob", 
    "prestige", 
    "highest_degree", 
    "sex_predicted", 
    "ethnicity_predicted", 
    "profile_linkedin_url", 
    "user_location", 
    "user_country", 
    "profile_title", 
    "updated_dt", 
    "numconnections", 
    "profile_summary",
    "updated" )


user_default= (0, # "user_id",
               "none", # "firstname",
                "none", # "lastname",
                "none", # "fullname",
                0.0, # "f_prob", 
                0.0, # "m_prob", 
                0.0, # "white_prob", 
                0.0, # "black_prob", 
                0.0, # "api_prob", 
                0.0, # "hispanic_prob", 
                0.0, # "native_prob", 
                0.0, # "multiple_prob", 
                0.0, # "prestige", 
                "none", # "highest_degree", 
                "none", # "sex_predicted", 
                "none", # "ethnicity_predicted", 
                "none", # "profile_linkedin_url", 
                "none", # "user_location", 
                "none", # "user_country", 
                "none", # "profile_title", 
                "none", # "updated_dt", 
                0, # "numconnections", 
                "none", # "profile_summary",
                False)# "updated" )

dt1 = {field_name: rg.make_default_list(field_default, 1, 100000) for (field_name, field_default) in itertools.zip_longest(user_var, user_default)}
df1 = [pd.DataFrame(dt1), pd.DataFrame(dt1), pd.DataFrame(dt1), pd.DataFrame(dt1)]

dt2 = {field_name: rg.make_default_list(field_default, 4, 100000) for (field_name, field_default) in itertools.zip_longest(user_var, user_default)}
df2 = pd.DataFrame(dt2)

In [ ]:
print(df2.at[1113, "user_id"][0])

In [ ]:
import sys

print(sys.getsizeof(df1[0]) + sys.getsizeof(df1[1]) + sys.getsizeof(df1[2]) + sys.getsizeof(df1[3]) )  

print(sys.getsizeof(df2))

# column_names = df2.columns.to_list()

# column_names.remove("user_id")

# print(column_names)

In [ ]:
# test sets
tset = {1, 5, 3, 4, 2}
s = pd.Series(list(tset))
print(s)